In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [2]:
import requests
import random
from itertools import cycle
import traceback

user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]


In [3]:
IP_start = pd.read_csv("/home/russell/Documents/DataScience/DonorsChoose/IPs/eighty_1.csv",header=None)
proxies = IP_start.iloc[:,0].values.tolist()
random.shuffle(proxies)
proxy_pool = cycle(proxies)

In [4]:
import pickle #for saving output files, pickles
import time #for time.sleep function to delay calls
from tqdm import tqdm #for updating loop
#from os import listdir
#from os.path import isfile, join
import glob #pattern matching and expansion.

In [5]:
def proposal_process(proplist):
    '''takes a list with proposal info and processes some info into a df '''
    #we can use values in these keys 'as is'
    keeperkeys = ['id', 'proposalURL', 'fundURL', 'imageURL', 'retinaImageURL', 'thumbImageURL', 'fulfillmentTrailer', 'percentFunded', 'numDonors', 'costToComplete', 'studentLed', 'numStudents', 'professionalDevelopment', 'distanceLearningProject', 'totalPrice', 'freeShipping', 'teacherId', 'teacherName', 'teacherTypes', 'schoolTypes', 'schoolName', 'schoolUrl', 'city', 'zip', 'state', 'stateFullName', 'latitude', 'longitude', 'expirationDate', 'expirationTime', 'fundingStatus', 'fullyFundedDate', 'waitingForCheckPayment', 'modifiedDate','shortDescription']
    
    #these require more finagling
    moreprocess = ['matchingFund', 'gradeLevel', 'povertyType']
    
    combine=[]
    
    for prop in proplist:
        new_dict = {k: prop[k] for k in keeperkeys if k in prop}
        keepinfo = pd.DataFrame.from_dict(new_dict,orient='index').T
        
        if(not bool(prop['matchingFund'].get('matchingKey'))):
            keepinfo['matchingfund'] = 'no'
        else:
            keepinfo['matchingfund'] = 'no'
            
        keepinfo['grade_level'] = prop['gradeLevel']['name']
        keepinfo['poverty_level'] = prop['povertyType']['label']
        combine.append(keepinfo)
        
    summarydf = pd.concat(combine)
    return(summarydf)

In [6]:
# 1,674,610 completed projects 
prog_count = 0
iteration = 0
tenner = list(range(0,int(1674550/50),10))
#tenner = list(range(0,50,10))
historical = "https://www.donorschoose.org/common/json_feed.html?showFacetCounts=true&APIKey=H9v7hCeN&max=50&historical=true&index="
out_file = "/home/russell/Documents/DataScience/DonorsChoose/Data/Metrics/pickle_"

In [8]:
def pretty_print_POST(req):
    """
    At this point it is completely built and ready
    to be fired; it is "prepared".

    However pay attention at the formatting used in 
    this function because it is programmed to be pretty 
    printed and may differ from the actual request.
    """
    print('{}\n{}\r\n{}\r\n\r\n{}'.format(
        '-----------START-----------',
        req.method + ' ' + req.url,
        '\r\n'.join('{}: {}'.format(k, v) for k, v in req.headers.items()),
        req.body,
    ))

for m in range(100):
     #Get a proxy from the pool
    proxy = next(proxy_pool)
    
    #Pick a random user agent
    user_agent = random.choice(user_agent_list)
    #Set the headers 
    header_val = {'User-Agent': user_agent}
         
    #randomly pops element from tenner out (for use), and removes it from tenner so it isn't resampled    
    random_element_from_tenner = tenner.pop(random.randrange(len(tenner)))     
        
    page_to_use = historical + str(random_element_from_tenner) #change index value every iteration, references tenner list

    r = requests.post(page_to_use,proxies={"http": proxy, "https": proxy}, headers=header_val) #send request to specific html address
    prepared = r.prepare()
    pretty_print_POST(prepared)

KeyboardInterrupt: 

In [ ]:
  
while prog_count <= max(tenner):  #1674610
    
    #Get a proxy from the pool
    proxy = next(proxy_pool)
    
    #Pick a random user agent
    user_agent = random.choice(user_agent_list)
    #Set the headers 
    header_val = {'User-Agent': user_agent}
         
    #randomly pops element from tenner out (for use), and removes it from tenner so it isn't resampled    
    random_element_from_tenner = tenner.pop(random.randrange(len(tenner)))     
        
    page_to_use = historical + str(random_element_from_tenner) #change index value every iteration, references tenner list

    #Make the request
    try:
        r = requests.get(page_to_use,proxies={"http": proxy, "https": proxy}, headers=header_val) #send request to specific html address
    except:
        tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
        continue #if we have an exception, *continue* will skip the rest of the iteration
    
    try:
        data_dict = json.loads(r.text) #load the text as a dictionary
    except: #catch *all* exceptions
        tenner.append(random_element_from_tenner)#re-attaches item pulled off, to try again later
        continue #if we have an exception, *continue* will skip the rest of the iteration
    
    dicts_on_this_page = list(data_dict.values())
    proposal_ind = ([list(data_dict.keys()).index('proposals')])[0]  #
    proposal_list = dicts_on_this_page[proposal_ind]

    outframe = proposal_process(proposal_list)
    #print(outframe['teacherName'])
    pickle_out = open(out_file+("{0:07}".format(iteration))+'.pickle',"wb")
    pickle.dump(outframe, pickle_out)
    pickle_out.close()
    iteration += 1
    prog_count += 50
    print("Current progress:",np.round(prog_count/(max(tenner))),"%")
    time.sleep(random.randint(5,300))